In [1]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm

import sys
sys.path.append( '../' )
import importlib

import measures_common
importlib.reload(measures_common)

<module 'measures_common' from 'd:\\Projetos\\TCC\\Part_2.1-Complex_Networks\\adjacency\\..\\measures_common.py'>

In [4]:
def compare_measures(measures, fake_text, true_text):

    fake_measures = measures_common.get_measures_from_text(fake_text)
    true_measures = measures_common.get_measures_from_text(true_text)

    for key in fake_measures.keys():
        if true_measures[key] > fake_measures[key]:
            measures[key] += 1
        
    
def get_comparison_measures_df(df):

    measures = defaultdict(int)
    measures_percentage = defaultdict(float)

    for index, row in tqdm(df.iterrows(), total=len(df), unit='rows', mininterval=1 ,desc='Comparing measures from dataframe'):
        compare_measures(measures, row['Fake'], row['True'])

    for key in list(measures.keys()):
        measures_percentage[key] = round(measures[key]/len(df), 2)


    return measures, measures_percentage

In [5]:
file_path_with_stopwords = os.path.join('..', '..', 'data', 'csvs', 'fake_vs_true_pre_norm_with_stopwords.csv')
file_path_without_stopwords = os.path.join('..', '..', 'data', 'csvs', 'fake_vs_true_pre_norm.csv')

df_with_stopwords = pd.read_csv(file_path_with_stopwords)
df_without_stopwords = pd.read_csv(file_path_without_stopwords)

df_with_stopwords.head(3)

,Fake,True
0,katia abreu diz que vai colocar sua expulsao e...,o podemos decidiu expulsar o deputado federal ...
1,dr ray peita bolsonaro chamao de conservador f...,bolsonaro e um liberal completo diz presidente...
2,reinaldo azevedo desmascarado pela policia fed...,ministro do stf libera andrea neves de prisao ...


In [ ]:
measures_with_stopwords, measures_percentage_with_stopwords = get_comparison_measures_df(df_with_stopwords)
measures_without_stopwords, measures_percentage_without_stopwords = get_comparison_measures_df(df_without_stopwords)

In [76]:
cols = pd.MultiIndex.from_product([["with stopwords","without stopwords"], ['true>fake', '%']])

df_metrics = pd.DataFrame(np.array([list(measures_with_stopwords.values()), list(measures_percentage_with_stopwords.values()),
                                    list(measures_without_stopwords.values()), list(measures_percentage_without_stopwords.values())]).T,
                            columns=cols, 
                            index=measures_with_stopwords.keys())

In [3]:
df_metrics = df_metrics.reset_index(col_level=1).rename(columns={'index':'Measure'})
df_metrics.to_csv("../results/compare_metrics_adjacency.csv", index=False)

In [6]:
df_metrics = pd.read_csv("../results/compare_metrics_adjacency.csv", header=[0,1], index_col=0)
df_metrics.style.highlight_min(axis = 0, props='color:lightgreen;').highlight_max(axis = 0, props='color:lightgreen;').format(precision=2)